In [66]:
import numpy as np
import pandas as pd
import xarray as xr
import os 
from itertools import product
import matplotlib.pyplot as plt
import glob
import datetime

In [7]:
os.getcwd()

'/home/student/Dokumente/moonsoon'

In [8]:
src = os.path.join(os.getcwd(), 'moonsoon_src')

SPECS

In [53]:
colspecs = [(0,6),(6,10),(10,14),(14,20),(20,26),(26,32),(32,38),(38,44),(44,-1)]
colnames = ['year','month','day','tas','tas_max','tas_min','pre','v8','v9']

#READ FILE
year, month, day, tas, tas_max, tas_min, 

In [55]:
path = os.path.join(src, '*.dly')
files = glob.glob(path)
print(files[:1])
for filepath in files[:1]:
    df = pd.read_fwf(filepath, colspecs=colspecs, names= colnames)


['/home/student/Dokumente/moonsoon/moonsoon_src/5924_M_5.dly']


In [77]:
int(datetime.date(2015,1,1).strftime('%j'))

1

In [91]:
df

,year,month,day,tas,tas_max,tas_min,pre,v8,v9
0,2015,1,1,15.26,31.27,13.40,0.00,0.61,2.04
1,2015,1,2,14.70,27.56,13.16,0.00,0.67,2.45
2,2015,1,3,15.25,26.02,9.12,0.00,0.70,1.56
3,2015,1,4,14.97,23.79,8.89,0.00,0.73,0.91
4,2015,1,5,14.71,23.15,7.91,0.00,0.72,1.59
...,...,...,...,...,...,...,...,...,...
31406,2100,12,27,14.95,33.27,17.53,0.00,0.60,2.19
31407,2100,12,28,15.04,34.32,17.77,0.00,0.57,2.60
31408,2100,12,29,14.98,34.36,19.70,0.00,0.57,2.33
31409,2100,12,30,14.80,34.34,19.82,0.81,0.57,2.06


In [109]:
df['date'] = pd.to_datetime(df[['year','month','day']],yearfirst=True, format = "%j", exact = True)

In [111]:
df['dayoftheyear'] = df['date'].dt.strftime('%j')

In [112]:
df

,year,month,day,tas,tas_max,tas_min,pre,v8,v9,dayoftheyear,date
0,2015,1,1,15.26,31.27,13.40,0.00,0.61,2.04,001,2015-01-01
1,2015,1,2,14.70,27.56,13.16,0.00,0.67,2.45,002,2015-01-02
2,2015,1,3,15.25,26.02,9.12,0.00,0.70,1.56,003,2015-01-03
3,2015,1,4,14.97,23.79,8.89,0.00,0.73,0.91,004,2015-01-04
4,2015,1,5,14.71,23.15,7.91,0.00,0.72,1.59,005,2015-01-05
...,...,...,...,...,...,...,...,...,...,...,...
31406,2100,12,27,14.95,33.27,17.53,0.00,0.60,2.19,361,2100-12-27
31407,2100,12,28,15.04,34.32,17.77,0.00,0.57,2.60,362,2100-12-28
31408,2100,12,29,14.98,34.36,19.70,0.00,0.57,2.33,363,2100-12-29
31409,2100,12,30,14.80,34.34,19.82,0.81,0.57,2.06,364,2100-12-30


In [141]:
def moonsoon(df,streak_condition = 8):
    final_first_day = False
    final_last_day = False
    current_first_day = -3
    current_day = -5
    streak = 0
    count = 0
    for index, row in df.iterrows():
        current_day = row.dayoftheyear
        #if there is no rain in the current day: set everything as default
        if row.pre == 0:
            current_first_day = -3
            streak = 0
        #if the day rains and there is no rain the day before (streak = 0)
        #:save the current first day as current day and steak +1
        elif row.pre != 0  and streak == 0:
            current_first_day = current_day
            streak += 1
        #if there is rain and also yesterday: just set streak +1
        elif row.pre !=0 and streak !=0:
            streak += 1
        #if the current streak = streak_condition and there is no first day yet, take it as final first day.
        if streak == streak_condition and final_first_day is False:
            final_first_day = current_first_day
        #if the current streak = streak_condition, save it to the  final_last_day.
        if streak >= streak_condition:
            final_last_day = current_day
    return (final_first_day, final_last_day)

In [142]:
moonsoon_value = []
for year in df['year'].unique() :
    fday, lday = moonsoon(df[df.year == year])
    moonsoon_value.append([year, fday, lday])


In [143]:
moonsoon_df = pd.DataFrame(data= moonsoon_value, columns = ['year','firstday','lastday'])


In [144]:
moonsoon_df.to_csv('/home/student/Dokumente/moonsoon/moonsoon_src/test.txt', index=False)

In [153]:
'%s' % streak_condition

'8'

In [160]:
for streak_condition in [6,7,9,10,11]:
    for filepath in files:
        #formulate the paths
        directory, filename = os.path.split(filepath)
        new_filename = filename[:-3] + 'txt'
        output_dir = os.path.join(directory,'moonsoon with streak of %s'%streak_condition)
        output_path = os.path.join(output_dir,new_filename)
        #create dir if new
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        #read file and add column day of the year
        df = pd.read_fwf(filepath, colspecs=colspecs, names= colnames)
        df['date'] = pd.to_datetime(df[['year','month','day']],yearfirst=True, format = "%j", exact = True)
        df['dayoftheyear'] = df['date'].dt.strftime('%j')
        #processing data
        moonsoon_value = []
        for year in df['year'].unique() :
            fday, lday = moonsoon(df[df.year == year], streak_condition = streak_condition)
            moonsoon_value.append([year, fday, lday])
        #save as df and to csv
        moonsoon_df = pd.DataFrame(data= moonsoon_value, columns = ['year','firstday','lastday'])
        moonsoon_df.to_csv(output_path, index=False)
    

In [146]:
output_path

'/home/student/Dokumente/moonsoon/moonsoon_src/5924_M_5..txt'